In [36]:
%run greedy_search.ipynb
%run brute_force.ipynb
%run variable_neighborhood_search.ipynb

In [37]:
import re
import random
import os


In [38]:
def find_common_ones(set_of_models):
    n=len(set_of_models[0])
    common_ones=[]

    for i in range(n):
        all_ones=True
        for var in set_of_models:
            if var[i] != 1:
                all_ones=False
                break
        if all_ones:
            common_ones.append(i)
    
    return common_ones

def shorten_models(set_of_models,common_ones):
    shortened_models = []

    for var in set_of_models:
        shortened_var=''.join([str(var[i]) for i in range(len(var)) if i not in common_ones])
        shortened_models.append((var,shortened_var))
    
    return shortened_models

def generate_binary_combinations(n):
    if n == 1:
        return ["0", "1"]
    else:
        smaller_combinations = generate_binary_combinations(n - 1)
        return ["0" + combination for combination in smaller_combinations] + ["1" + combination for combination in smaller_combinations]

def generate_all_combinations(n,common_ones):
    all_combinations=generate_binary_combinations(n)
    
    duzina=len(all_combinations[0])+len(common_ones)
    extended_numbers=[]

    for i in range(len(all_combinations)):
        el_i=0
        extended_number=""
        for j in range(duzina):
            if j in common_ones:
                extended_number+="1"
            else:
                extended_number+=all_combinations[i][el_i]
                el_i+=1
        extended_numbers.append((extended_number,all_combinations[i]))

    return extended_numbers


def generate_formula(set_of_models):
    common_ones=find_common_ones(set_of_models)
    shortened_models=shorten_models(set_of_models,common_ones)
    all_models=generate_all_combinations(len(shortened_models[0][1]),common_ones)

    formula=""

    for i in range(len(all_models)):
        if all_models[i][1] not in [model[1] for model in shortened_models]:
            cnf_clauses = []
            for j, bit  in enumerate(all_models[i][0]):
                if j not in common_ones:
                    if bit == "1":
                        cnf_clauses.append(f"~x{j+1}")
                    else:
                        cnf_clauses.append(f"x{j+1}")

            formula += "(" + " | ".join(cnf_clauses) + ") & "
        
    for i in common_ones:
        formula += f"x{i+1} & "
    
    formula = formula[:-3]
    return formula

In [39]:
def count_positive_literals(clause):
    literals=re.findall(r'(~?\w+)',clause)
    count = 0
    for literal in literals:
        if '~' not in literal:
            count += 1
    return count

def exists_non_horn_clause(clauses):
    for clause in clauses:
        if count_positive_literals(clause)>1:
            return True
    return False

def can_be_resolved(literals1,literals2):
    symm_diff=literals1.symmetric_difference(literals2)

    complement_literals_counter=0

    for literal in symm_diff:
        if literal in literals1 and '~' + literal in literals2:
            complement_literals_counter+=1
        elif literal in literals2 and '~' + literal in literals1:
            complement_literals_counter+=1


    if complement_literals_counter != 1:
        return 0
    else:
        return 1


def reduce_formula(formula):
    clauses=list(formula.split("&"))
    original_clauses=clauses.copy()

    while exists_non_horn_clause(clauses):
        resolved_clauses=set()
        resolved_clause=None
        resolved=False
        for i in range(len(clauses)):
            for j in range(i + 1, len(clauses)):
                clause1 = clauses[i]
                clause2 = clauses[j]
                resolved_clause,need_clear= resolve_clause(clause1,clause2)
                if resolved_clause and need_clear:
                    if resolved_clause not in clauses:
                        resolved_clauses.add(resolved_clause)
                    resolved=True
                    break
            if resolved:
                break
                
        if not resolved:
            clauses=original_clauses.copy()
            random.shuffle(clauses)

        if need_clear:
            clauses.remove(clause1)
            clauses.remove(clause2)
        
        for resolved_clause in resolved_clauses:
            clauses.append(resolved_clause)
                                
    return " & ".join(clauses)

def resolve_clause(clause1,clause2):
    literals1=set(re.findall(r'(~?\w+)',clause1))
    literals2=set(re.findall(r'(~?\w+)',clause2))
    need_clear=True

    if len(literals1)!=len(literals2) or (len(literals1) == len(literals2) and len(literals1.intersection(literals2)) != len(literals1)-1):
        need_clear=False


    if not can_be_resolved(literals1,literals2):
        return None,False

    resolved_literal=None
    resolved=False

    for literal1 in literals1:
        for literal2 in literals2:
            if literal1=='~' + literal2 or literal2 == '~' + literal1:
                resolved_literal = literals1.union(literals2) - {literal1} - {literal2}
                resolved=True
                break
        if resolved:
            break

    if resolved:
        return "(" + " | ".join(resolved_literal) + ")",need_clear
    else:
        return None,False

In [40]:
def load_from_file(filename):
    M=[]
    n=0
    with open(filename,"r") as file:
        lines=file.readlines()
        for line in lines:
            line=line.strip()
            values=tuple(int(x) for x in line.split())
            if n==0:
                n=len(values)
            M.append(values)
    return M,n

In [41]:
def test_bf(M,n):
    max_core=brute_force(M,n)
    print("Brute force results: ")
    print("Number of assignments in Horn core:",len(max_core))
    print("Maximal Horn Core:",max_core)
    formula=generate_formula(max_core)
    print("Formula:",formula)
    print("Horn formula:",reduce_formula(formula))


In [42]:
def test_gs(M,n):
    max_core,_=greedy_search(M,n)
    print("Greedy search results: ")
    print("Number of assignments in Horn core:",len(max_core))
    print("Maximal Horn Core:",max_core)
    formula=generate_formula(max_core)
    print("Formula:",formula)
    print("Horn formula:",reduce_formula(formula))

In [43]:
def test_vns(M,n):
    max_core,_=variable_neighborhood_search(M,50,5,0.5)
    print("Variable neighborhood results: ")
    print("Number of assignments in Horn core:",len(max_core))
    print("Maximal Horn Core:",max_core)
    formula=generate_formula(max_core)
    print("Formula:",formula)
    print("Horn formula:",reduce_formula(formula))

In [44]:
def test_all(M,n):
    print("Number of assignments:",len(M))
    print("M:",M)
    print("")
    test_bf(M,n)
    print("")
    test_gs(M,n)
    print("")
    test_vns(M,n)

In [45]:
M,n=load_from_file("./TestInstances/TestInstances1.txt")
test_all(M,n)

Number of assignments: 4
M: [(0, 0, 0, 1), (0, 1, 0, 0), (1, 0, 0, 1), (0, 1, 1, 1)]

Brute force results: 
Number of assignments in Horn core: 3
Maximal Horn Core: [(0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 1, 1)]
Formula: (x1 | x2 | ~x3) & (x1 | ~x2 | x3) & (~x1 | x2 | ~x3) & (~x1 | ~x2 | x3) & (~x1 | ~x2 | ~x3) & x4
Horn formula:  (~x1 | ~x2 | ~x3)  &  x4 & (x2 | ~x3) & (~x2 | x3)

Greedy search results: 
Number of assignments in Horn core: 3
Maximal Horn Core: [(0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 1, 1)]
Formula: (x1 | x2 | ~x3) & (x1 | ~x2 | x3) & (~x1 | x2 | ~x3) & (~x1 | ~x2 | x3) & (~x1 | ~x2 | ~x3) & x4
Horn formula:  (~x1 | ~x2 | ~x3)  &  x4 & (x2 | ~x3) & (~x2 | x3)

Variable neighborhood results: 
Number of assignments in Horn core: 3
Maximal Horn Core: [(0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 1, 1)]
Formula: (x1 | x2 | ~x3) & (x1 | ~x2 | x3) & (~x1 | x2 | ~x3) & (~x1 | ~x2 | x3) & (~x1 | ~x2 | ~x3) & x4
Horn formula:  (~x1 | ~x2 | ~x3)  &  x4 & (x2 | ~x3) & (~x2 | x3)


In [46]:
M,n=load_from_file("./TestInstances/TestInstances2.txt")
test_all(M,n)

Number of assignments: 10
M: [(0, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 0), (0, 1, 0, 0), (0, 1, 1, 0), (1, 0, 1, 0), (0, 0, 0, 0), (1, 0, 0, 1), (1, 0, 0, 0), (0, 0, 1, 1)]

Brute force results: 
Number of assignments in Horn core: 10
Maximal Horn Core: [(0, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 0), (0, 1, 0, 0), (0, 1, 1, 0), (1, 0, 1, 0), (0, 0, 0, 0), (1, 0, 0, 1), (1, 0, 0, 0), (0, 0, 1, 1)]
Formula: (x1 | ~x2 | ~x3 | ~x4) & (~x1 | x2 | ~x3 | ~x4) & (~x1 | ~x2 | x3 | x4) & (~x1 | ~x2 | x3 | ~x4) & (~x1 | ~x2 | ~x3 | x4) & (~x1 | ~x2 | ~x3 | ~x4)
Horn formula:  (~x1 | x2 | ~x3 | ~x4)  &  (~x1 | ~x2 | ~x3 | x4)  & (~x2 | ~x4 | ~x3) & (~x2 | x3 | ~x1)

Greedy search results: 
Number of assignments in Horn core: 10
Maximal Horn Core: [(0, 0, 0, 1), (0, 0, 1, 0), (0, 1, 0, 1), (0, 1, 0, 0), (0, 1, 1, 0), (1, 0, 1, 0), (0, 0, 0, 0), (1, 0, 0, 1), (1, 0, 0, 0), (0, 0, 1, 1)]
Formula: (x1 | ~x2 | ~x3 | ~x4) & (~x1 | x2 | ~x3 | ~x4) & (~x1 | ~x2 | x3 | x4) & (~x1 | ~x2 | x3 | ~x4) & (~x1 | ~x2 | ~